# ANC Ollama: Wikipedia + FAISS + Ollama (Databricks-Parity)

This notebook mirrors the `Config` and `Wikipedia Data Loader` sections from
`docs/ideation-kernel/01_agentic_wikipedia_aimpoint_interview.ipynb`, but swaps
Databricks components for local Ollama components on macOS.

Provider mapping used here:
- `ChatDatabricks` -> `ChatOllama`
- `DatabricksEmbeddings` -> `OllamaEmbeddings`
- Same `WikipediaLoader` + `FAISS` retrieval flow


## PlantUML Workflow Diagram

```plantuml
@startuml
title ANC Ollama Databricks-Parity Flow
start
:Define config;
:Load Wikipedia docs;
:Embed docs with OllamaEmbeddings;
:Index docs in FAISS;
:Run similarity search;
:Answer with ChatOllama;
stop
@enduml
```


In [7]:
# Uncomment in fresh environments:
# %pip install -q -r ../requirements-ollama-dev.txt


In [8]:
#######################################################################################################
###### Python Package Imports for this notebook                                                  ######
#######################################################################################################

# LangChain moved WikipediaLoader in newer releases; keep backward compatibility.
try:
    from langchain_community.document_loaders import WikipediaLoader
except ImportError:
    from langchain.document_loaders import WikipediaLoader

import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_ollama import ChatOllama, OllamaEmbeddings


#######################################################################################################
###### Config (Define LLMs, Embeddings, Vector Store, Data Loader specs)                         ######
#######################################################################################################

# DataLoader Config
query_terms = [
    "roadcut",
    "geology",
    "sedimentary rock",
    "stratigraphy",
]
max_docs = 10  # Start small while iterating locally.

# Retriever Config
k = 2
EMBEDDING_MODEL = "nomic-embed-text"
OLLAMA_BASE_URL = "http://localhost:11434"

# LLM Config
LLM_MODEL = "llama3.1:8b"
TEMPERATURE = 0.0

example_question = "What is a roadcut in geology?"


ModuleNotFoundError: No module named 'langchain'

In [ ]:
#######################################################################################################
###### Wikipedia Data Loader                                                                     ######
#######################################################################################################

query = " ".join(query_terms) if isinstance(query_terms, list) else query_terms
docs = WikipediaLoader(query=query, load_max_docs=max_docs).load()
print(f"Loaded {len(docs)} Wikipedia documents for query: {query!r}")

#######################################################################################################
###### FAISS Retriever: Using Ollama embedding model                                             ######
#######################################################################################################

# Define embeddings and the FAISS vector store.
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL, base_url=OLLAMA_BASE_URL)
vector_store = FAISS.from_documents(docs, embeddings)

# Example of how to invoke the vector store.
results = vector_store.similarity_search(example_question, k=k)
for i, res in enumerate(results, start=1):
    title = res.metadata.get("title", "unknown")
    source = res.metadata.get("source", "n/a")
    snippet = res.page_content[:220].replace("\n", " ")
    print(f"{i}. {title} ({source})")
    print(f"   {snippet}...")

#######################################################################################################
###### LLM: Using local Ollama model                                                             ######
#######################################################################################################

llm = ChatOllama(model=LLM_MODEL, base_url=OLLAMA_BASE_URL, temperature=TEMPERATURE)
response = llm.invoke(
    f"Answer concisely using Wikipedia-grounded context. Question: {example_question}"
)
print("\n", response.content)


## Local Ollama Prerequisites (macOS)

- Ensure Ollama is installed and running: `ollama serve`
- Pull the embedding model: `ollama pull nomic-embed-text`
- Pull the chat model: `ollama pull llama3.1:8b`
- Keep `notebooks/anc_dbrx.ipynb` for Databricks-specific libraries and endpoints
- Keep `notebooks/anc_gcp.ipynb` for Vertex AI-specific libraries and endpoints
